# 

In [1]:
from parameter_tuning import train_model
from rnn_layers import PredictTime
import torch
import torch.optim as optim
import torch.nn as nn
from parse_data import get_data, get_modified_values, get_binary_values, make_data_scalar

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device: ", device)


# Hyperparameters
sequence_length = [10,15,50,100,200]
hidden_layers = [1, 2, 5]
hidden_size = [10, 25, 50, 100]
lr = [0.001, 0.01, 0.1, 0.005, 0.05,0.5]
epochs = 100
optimizer = [optim.Adam]

Using device:  cpu


In [2]:
def make_data(df, device):

    x_train, y_train = [], []
    prev = None

    for row in df:
        if prev is None:
            prev = row
        x_train.append(prev)
        y_train.append(row)
        prev = row
    print(x_train[0].size())
    return torch.stack(x_train).float().to(device),torch.stack(y_train).float().to(device)

make_data(get_binary_values(get_data()), device)

torch.Size([1])


(tensor([[1.],
         [1.],
         [0.],
         ...,
         [1.],
         [0.],
         [1.]]),
 tensor([[1.],
         [0.],
         [1.],
         ...,
         [0.],
         [1.],
         [1.]]))

In [3]:
import random

options = []

for seq_len in sequence_length:
    for layers in hidden_layers:
        for hidden in hidden_size:
            for forcing in [True, False]:
                for l in lr:
                    entry = {}
                    entry["seq_len"] = seq_len
                    entry["layers"] = layers
                    entry["hidden"] = hidden
                    entry["forcing"] = forcing
                    entry["l"] = l
                    options.append(entry)
                
                                         
random.shuffle(options)    


In [ ]:

x_d, y_d = make_data(get_binary_values(get_data()), device)

best_model = None
best_history = None
best_score = 100000000000000
for entry in options:
    model = PredictTime(input_size=x_d[0].size()[0],
                        output_size=y_d[0].size()[0],
                        hidden_layers=entry["layers"],
                        hidden_size=entry["hidden"],
                        device=device).to(device)
    model, history = train_model(model=model,
                                 x_data=x_d,
                                 y_data=y_d,
                                 sequence_length=entry["seq_len"],
                                 epochs=epochs,
                                 loss=nn.BCELoss(),
                                 optimizer=optim.Adam,
                                 strict_teacher_forcing=entry["forcing"])
    if history[-1][1] < best_score:
        print("New best model:\nNew loss: ", history[-1], "\nOld loss:", best_history[-1])
        best_model = model
        best_history = history
        best_score = history[-1][1]
    else:
        print("Old model still stands:\Current loss: ", history[-1], "\Best loss:", best_history[-1])
                                 

    

torch.Size([1])
